In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
from coeditor.dataset import *
from coeditor.encoding import *
from coeditor.encoders import BasicQueryEditEncoder

from spot.utils import pretty_print_dict

os.chdir(proj_root())

In [2]:
# test_data_name = "medium"
# test_data_name = "SPOT"
test_data_name = "small"
# encoder = AnalysisBasedEditEncoder(extra_ctx_names=("usees", "post-usees"), add_truncate_bos=False)
# encoder = CstBasedEditEncoder(500, add_truncate_bos=False)
encoder = BasicQueryEditEncoder()
data = dataset_from_projects([proj_root()], encoder, [False], max_history_per_repo=10)

Create tokenized edits: 100%|██████████| 1/1 [00:03<00:00,  3.32s/chunk]


In [8]:
data = load_datasets(get_dataset_dir("small") / "BasicQueryEditEncoder(VERSION=2)", splits=["test"])["test"]

In [9]:
pretty_print_dict(data.overall_stats())

n_projects: 2
n_edits: 2039
n_additions: 0
input_tks:
   mean: 230.32
   median: 182
   min: 28
   max: 512
output_tks:
   mean: 57.2
   median: 43
   min: 4
   max: 256
references:
   mean: 75.127
   median: 8
   min: 1
   max: 877
ref_size_sum:
   mean: 17824
   median: 1345
   min: 43
   max: 467294


In [14]:
qedit = max(data.all_edits(), key=lambda e: e.tk_pedit.stats["ref_size_sum"])
print(qedit.show())

n_references: 778
n_ref_blocks: 2381
========Ground Truth========
 <1>:<add>         decoded_body = extract_params(r.data)
     <del>         decoded_body = extract_params(r.data)     
 <6>:<add>                     unicode(r.full_url), unicode(r.method), None, r.headers)
     <del>                     unicode(r.url), unicode(r.method), None, r.headers)
<10>:<add>                     unicode(r.full_url), unicode(r.method), r.data, r.headers)
     <del>                     unicode(r.url), unicode(r.method), r.data, r.headers)
<13>:<add>             if u_header in r.headers:
     <del>             if u_header in r.headers:               
<18>:<add>         return r
     <del>             return r

========Main Code========
# path: requests.auth/OAuth1.__call__

class OAuth1(AuthBase):
    def __call__(self, r):
 <0>        contenttype = r.headers.get('Content-Type', None)
 <1>        decoded_body = extract_params(r.data)     
 <2>        if contenttype == None and decoded_body!= None:
 <

In [10]:
call_exs = [ex for ex in data.all_edits() if ex.updated_calls]
print(f"Edits with updated calls: {len(call_exs)} / {len(data.all_edits())}")
print(f"Number of updated calls: {sum(len(ex.updated_calls) for ex in call_exs)}")

Edits with updated calls: 23 / 126
Number of updated calls: 30


In [ ]:
ex=call_exs[1]
print(f"updated calls: {[p for p, _ in ex.updated_calls]}")
print(ex.show())

In [17]:
# dataset_name = "SPOT"
dataset_name = "medium"
# encoder = AnalysisBasedEditEncoder(extra_ctx_names=("usees", "post-usees"))
encoder = CstBasedEditEncoder()
save_dir = get_dataset_dir(dataset_name) / repr_modified_args(encoder)
datasets = load_datasets(save_dir)

In [3]:
for group, dataset in datasets.items():
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

==================== test ====================
n_projects: 20
n_edits: 12666
input_size:
   mean: 2759.8
   median: 3137.5
   min: 44
   max: 4006
output_size:
   mean: 113.73
   median: 75
   min: 3
   max: 4776
==================== valid ====================
n_projects: 10
n_edits: 2297
input_size:
   mean: 2902.8
   median: 3629
   min: 58
   max: 4006
output_size:
   mean: 116.93
   median: 89
   min: 5
   max: 1567
==================== train ====================
n_projects: 100
n_edits: 50801
input_size:
   mean: 2579.5
   median: 2720
   min: 37
   max: 4006
output_size:
   mean: 98.522
   median: 64
   min: 3
   max: 8434


In [7]:
with_ctx = [e for e in data.all_edits() if e.extra_tks]

print("All edits:", len(data.all_edits()))
print("All edits with extra context:", len(with_ctx))

All edits: 1331
All edits with extra context: 360


In [11]:
from coeditor.model import DatasetDecodingResult

model_name = "coeditor-medium-analysis-post_usees"
dec_result: DatasetDecodingResult = pickle_load(get_model_dir() / model_name / "dec_result.pkl")

In [12]:
max_saved_samples = 200
random.seed(42)
exs_to_save = list(range(len(dec_result.predictions)))
random.shuffle(exs_to_save)
exs_to_save = exs_to_save[:max_saved_samples]
out_dir = get_model_dir() / model_name / "pred_samples"
dec_result.save_examples_to_dir(out_dir, exs_to_save)
print("Output examples saved to:", out_dir)

saving examples: 100%|██████████| 200/200 [00:03<00:00, 57.31it/s]

Output examples saved to: /mnt/nas/jiayi/coeditor/models/models/trained/coeditor-medium-analysis-post_usees/pred_samples
